In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("train.csv")
df

데이터 분석

datetime 연 월 일 시 나눠서 count 비교

season count 비교 -> 겨울은 적고 봄 여름이 많을걸로 예상

holiday count 비교, workingday와 다른 패턴이지 않을까 예상

workingday count 비교, 아침 저녁 많을듯

weather 1이면 화창, 숫자가 커지면 폭우,폭설 -> 1이 가장 많고 숫자가 크면 낮을것으로 예상

temp 실제 섭씨 온도 -> 적당한 온도 범위내에서(15~25도) 많고 그이상 이하이면 낮을것으로 예상

atemp 체감 섭씨 온도, 높으면 덥다고 낮으면 춥다고 느낌, temp와 비슷할거로 예상, 두개가 너무 비슷하면 하나 지우기

humidity 습도, 크면 짜증 -> 일정 수준을 넘어가면 불쾌지수가 높아지니까 낮게 예상

windspeed 풍속, 크면 바람 많이부는날 -> 일정 풍속을 넘어서면 count 낮게 예상

casual 비등록 대여 수

registered 등록 대여 수

count 비등록+등록 대여 총합, Target, casual+registered = count이므로 두개 삭제, 전체 수요예측이니까

In [ ]:
df.drop(['casual', 'registered'], axis=1, inplace=True)
df.info()

datetime 형변환 필요

In [ ]:
# datetime열 object에서 datetime type으로 변환, 분리
df['datetime'] = pd.to_datetime(df['datetime'])
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['hour'] = df['datetime'].dt.hour

In [ ]:
plt.xticks(rotation=45)
sns.boxplot(x='year', y='count', data = df)

df['year'].value_counts()

In [ ]:
sns.boxplot(x='month', y='count', data = df)
df['month'].value_counts()

In [ ]:
sns.boxplot(x='day', y='count', data = df)

In [ ]:
df['day'].value_counts()

In [ ]:
plt.xticks(rotation=45)
sns.barplot(x='hour', y='count', data = df)

dat count를 plot 해봤을때 20~31이 없어서 value_count로 확인해보니 실제로없음.

test.csv에 들어있을것 같다.

year 증가 count 증가, 상관 많지는 않은듯

month 4 ~ 12 비슷, 1 ~ 3 살짝 낮음

day 별차이 없음

In [ ]:
# season count 비교
sns.boxplot(x='season', y='count', data=df)

In [ ]:
# holiday 분석
sns.boxplot(x='holiday', y='count', data=df)

In [ ]:
# workingday 분석
sns.barplot(x='workingday', y='count', data=df)

In [ ]:
# holiday 기준: hour별 평균 count
holiday_group = df[df['holiday'] == 1].groupby('hour')['count'].mean().reset_index()
holiday_group['type'] = 'Holiday'

# workingday 기준: hour별 평균 count
workingday_group = df[df['workingday'] == 1].groupby('hour')['count'].mean().reset_index()
workingday_group['type'] = 'Workingday'

# 두 데이터프레임 합치기
compare_df = pd.concat([holiday_group, workingday_group])

# 시각화
plt.figure(figsize=(12, 6))
sns.lineplot(data=compare_df, x='hour', y='count', hue='type', marker='o')
plt.title('Workingday vs Holiday')
plt.xlabel('Hour')
plt.ylabel('Average Count')
plt.grid(True)
plt.xticks(range(0, 24))
plt.legend(title='Day Type')
plt.show()

workingday holiday는 각각 보면 별 의미 없는것 같아 보이는데 같이 비교해보니 시간대별로 차이가 있음

삭제 X

In [ ]:
# weather 분석
sns.boxplot(x='weather', y='count', data=df)

In [ ]:
# weather 기준: hour별 평균 count
weather_group1 = df[df['weather'] == 1].groupby('hour')['count'].mean().reset_index()
weather_group1['type'] = '1'

weather_group2 = df[df['weather'] == 2].groupby('hour')['count'].mean().reset_index()
weather_group2['type'] = '2'

weather_group3 = df[df['weather'] == 3].groupby('hour')['count'].mean().reset_index()
weather_group3['type'] = '3'

weather_group4 = df[df['weather'] == 4].groupby('hour')['count'].mean().reset_index()
weather_group4['type'] = '4'

In [ ]:
compare_df = pd.concat([weather_group1, weather_group2, weather_group3, weather_group4])

# 시각화
plt.figure(figsize=(12, 6))
sns.lineplot(data=compare_df, x='hour', y='count', hue='type', marker='o')
plt.title('Weather')
plt.xlabel('Hour')
plt.ylabel('Average Count')
plt.grid(True)
plt.xticks(range(0, 24))
plt.legend(title='Day Type')
plt.show()

weather 4일때는 거의 타지 않는다

In [ ]:
# temp 분석
print(df['temp'].describe())

# 구간 나누기
bins = [0, 5, 10, 15, 20, 25, 30, 35, 42]  # 임의 범위 설정
labels = ['0', '1', '2', '3', '4','5','6','7']

# 구간 컬럼 생성
df['temp_bin'] = pd.cut(df['temp'], bins=bins, labels=labels)

plt.figure(figsize=(10, 5))
sns.barplot(x='temp_bin', y='count', data=df, order=labels)  # order로 순서 유지
plt.title('temp / Count')
plt.xlabel('temp Group')
plt.ylabel('Count')
plt.show()

In [ ]:
# atemp 분석
print(df['atemp'].describe())

# 구간 나누기
bins = [0,5,10,15,20,25,30,35,40,50]  # 임의의 넉넉한 범위 설정
labels = ['0', '1', '2', '3', '4','5','6','7','8']

# 구간 컬럼 생성
df['atemp_bin'] = pd.cut(df['atemp'], bins=bins, labels=labels)

plt.figure(figsize=(10, 5))
sns.barplot(x='atemp_bin', y='count', data=df, order=labels)  # order로 순서 유지
plt.title('atemp / Count')
plt.xlabel('atemp Group')
plt.ylabel('Count')
plt.show()

In [ ]:
# humidity 분석
print(df['humidity'].describe())

# 구간 나누기
bins = [-1,10,20,30,40,50,60,70,80,90,100]  # 임의의 넉넉한 범위 설정
labels = ['0', '1', '2', '3', '4','5','6','7','8','9']

# 구간 컬럼 생성
df['humidity_bin'] = pd.cut(df['humidity'], bins=bins, labels=labels)

plt.figure(figsize=(10, 5))
sns.barplot(x='humidity_bin', y='count', data=df, order=labels)  # order로 순서 유지
plt.title('humidity / Count')
plt.xlabel('humidity Group')
plt.ylabel('Count')
plt.show()

In [ ]:
# windspeed 분석
print(df['windspeed'].describe())

# 구간 나누기
bins = [-5,6,20,29,39,49,61]  # 임의 범위 설정
labels = ['0', '2', '4', '5', '6','7']

# 구간 컬럼 생성
df['windspeed_bin'] = pd.cut(df['windspeed'], bins=bins, labels=labels)

plt.figure(figsize=(10, 5))
sns.barplot(x='windspeed_bin', y='count', data=df, order=labels)  # order로 순서 유지
plt.title('windspeed / Count')
plt.xlabel('windspeed Group')
plt.ylabel('Count')
plt.show()

In [ ]:
df.drop(['datetime', 'atemp'], axis=1, inplace=True) # 삭제

In [ ]:
# 그룹별 평균 count 계산
grouped_hd = df.groupby(['holiday', 'hour'])['count'].mean().reset_index()

# 시각화
plt.figure(figsize=(10, 5))
sns.barplot(data=grouped_hd, x='hour', y='count', hue='holiday')
plt.title('Compare workingday and holiday')
plt.xlabel('Hour')
plt.ylabel('Count')
plt.legend(title='0 : workingday,\n1 : Holiday')
plt.grid(True)
plt.show()

test.csv 이후 결과 제출시 양식은 아래와 같다

datetime,count

2011-01-20 00:00:00,0

2011-01-20 01:00:00,0

2011-01-20 02:00:00,0

2011-01-20 03:00:00,0

2011-01-20 04:00:00,0

2011-01-20 05:00:00,0

2011-01-20 06:00:00,0